# Objective and Motivation

Agriculture plays a central role globally by providing food for humans and livestock or material for industrial processes. It is a driver of economic growth and will play an essential role in reducing the impact of climate change and greening our economies. Accurate and reliable agricultural data is therefore paramount to ensure food security and global subsistence.

<img src="ai4eo.png" alt="example image of input an output" title="Fig.1: Example of low resolution (10m) input image from Sentinental and high resolution (2.5m) binary output map" width="1500"/>

The goal of this challenge is to map cultivated land using Copernicus Sentinel imagery, and to develop solutions to extract as much information as possible from the native 10-meter per pixel resolution. The area of interest for this challenge is Slovenia and the training images' area are shown in Fig. 1. An example input image for one time step from Sentinel imagery and the corresponding high-resolution output map is shown in Fig. 1. 

**Task:** Estimate a cultivated land binary map at 2.5 metres spatial resolution given as input a Sentinel-2 time-series at 10 metres spatial resolution, therefore resulting in a 4x spatial resolution enhancement.

# Setup

The following cell contains all the modules needed and used in this notebook.

In [ ]:
# Built-in modules
import os
import json
import datetime as dt
from typing import Tuple, List
import argparse

# Basics of Python data handling and visualization
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from tqdm.auto import tqdm

# Math modules
from scipy.stats import skewnorm
import math

# Module for GeoDB
from xcube_geodb.core.geodb import GeoDBClient

# Imports from eo-learn and sentinelhub-py
from sentinelhub import CRS, BBox, SHConfig, DataCollection

from eolearn.core import (FeatureType,
                          EOPatch, 
                          EOTask, 
                          LinearWorkflow, 
                          EOExecutor, 
                          LoadTask,
                          SaveTask)
from eolearn.io import GeoDBVectorImportTask, SentinelHubInputTask, ExportToTiff
from eolearn.geometry import VectorToRaster
from eolearn.features import NormalizedDifferenceIndexTask, SimpleFilterTask
from eolearn.mask import AddValidDataMaskTask

# algorithms for image processing and computer vision
from skimage import measure
from skimage.morphology import binary_dilation, disk
from PIL import Image

# Modules for modelling
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

In [ ]:
%matplotlib inline

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# EOTasks

The following cell contains all EOTasks used. Some are taken from the starter notebook and some have been defined by ourselves.

In [ ]:
class SamplePatchletsTask(EOTask):
    '''
    Sample patchlets from EOTask
    '''

    SCALE_FACTOR = 4 # do not change

    def __init__(self, s2_patchlet_size: int, num_samples: int, random_mode: bool):
        """ Set-up of task 
        
        :param s2_patchlet_size: Size in pixels of resulting patchlet
        :param num_samples: Number of patchlets to sample
        """
        self.s2_patchlet_size = s2_patchlet_size
        self.num_samples = num_samples
        self.random_mode = random_mode

    def _calculate_sampled_bbox(self, bbox: BBox, r: int, c: int, s: int,
                                resolution: float) -> BBox:
        """ Calculate bounding box of smaller patchlets """
        return BBox(((bbox.min_x + resolution * c,  bbox.max_y - resolution * (r + s)),
                     (bbox.min_x + resolution * (c + s), bbox.max_y - resolution * r)),
                    bbox.crs)

    def _sample_s2(self, eop: EOPatch, row: int, col: int, size: int, 
                   resolution: float = 10):
        """ Randomly sample a patchlet from the EOPatch """
        # create a new eopatch for each sub-sample
        sampled_eop = EOPatch(timestamp=eop.timestamp, 
                              scalar=eop.scalar, 
                              meta_info=eop.meta_info)
        
        # sample S2-related arrays
        features = eop.get_feature_list()
        s2_features = [feature for feature in features 
                       if isinstance(feature, tuple) and 
                       (feature[0].is_spatial() and feature[0].is_time_dependent())]
        
        for feature in s2_features:
            sampled_eop[feature] = eop[feature][:, row:row + size, col:col + size, :]
        
        # calculate BBox for new sub-sample
        sampled_eop.bbox = self._calculate_sampled_bbox(eop.bbox, 
                                                        r=row, c=col, s=size, 
                                                        resolution=resolution)
        sampled_eop.meta_info['size_x'] = size
        sampled_eop.meta_info['size_y'] = size
        
        # sample from target maps, beware of `4x` scale factor
        target_features = eop.get_feature(FeatureType.MASK_TIMELESS).keys()
        
        for feat_name in target_features:
            sampled_eop.mask_timeless[feat_name] = \
            eop.mask_timeless[feat_name][self.SCALE_FACTOR*row:self.SCALE_FACTOR*row + self.SCALE_FACTOR*size, 
                                         self.SCALE_FACTOR*col:self.SCALE_FACTOR*col + self.SCALE_FACTOR*size]
        
        # sample from weight maps, beware of `4x` scale factor
        target_features = eop.get_feature(FeatureType.DATA_TIMELESS).keys()
        
        for feat_name in target_features:
            sampled_eop.data_timeless[feat_name] = \
            eop.data_timeless[feat_name][self.SCALE_FACTOR*row:self.SCALE_FACTOR*row + self.SCALE_FACTOR*size, 
                                         self.SCALE_FACTOR*col:self.SCALE_FACTOR*col + self.SCALE_FACTOR*size]
        
        return sampled_eop

    def execute(self, eopatch_s2: EOPatch, buffer: int=0,  seed: int=42, random_mode: bool=1) -> List[EOPatch]:
        """ Sample a number of patchlets from the larger EOPatch. 
        
        :param eopatch_s2: EOPatch from which patchlets are sampled
        :param buffer: Do not sample in a given buffer at the edges of the EOPatch
        :param seed: Seed to initialise the pseudo-random number generator
        :param random_mode: Select the upper left corner at random (default: True)
        """
        _, n_rows, n_cols, _ = eopatch_s2.data['BANDS'].shape
        np.random.seed(seed)
        eops_out = []
        
        if not self.random_mode:
            max_per_row = n_rows // self.s2_patchlet_size
            max_per_col = n_cols // self.s2_patchlet_size
        
        # random sampling of upper-left corner. Added: Change this for non-overlapping patchlets
        for patchlet_num in range(0, self.num_samples):
            if self.random_mode:
                row = np.random.randint(buffer, n_rows - self.s2_patchlet_size - buffer)
                col = np.random.randint(buffer, n_cols - self.s2_patchlet_size - buffer)
            else:
                row = (buffer + patchlet_num // int(np.floor((n_rows - buffer) / self.s2_patchlet_size)) * self.s2_patchlet_size)
                col = buffer + (patchlet_num * self.s2_patchlet_size) % (n_cols - buffer - self.s2_patchlet_size)
                
                row = (patchlet_num // max_per_row) * self.s2_patchlet_size
                col = (patchlet_num % max_per_col) * self.s2_patchlet_size
                
                
            sampled_s2 = self._sample_s2(eopatch_s2, row, col, self.s2_patchlet_size)
            eops_out.append(sampled_s2)

        return eops_out

class ComputeReflectances(EOTask):
    """ Apply normalisation factors to DNs (from starter notebook)"""
    def __init__(self, feature):
        self.feature = feature
        
    def execute(self, eopatch):
        eopatch[self.feature] = eopatch.scalar['NORM_FACTORS'][..., None, None] \
            * eopatch[self.feature].astype(np.float32)
        return eopatch

class SentinelHubValidData:
    """
    Combine 'CLM' mask with `IS_DATA` to define a `VALID_DATA_SH` mask
    The SentinelHub's cloud mask is asumed to be found in eopatch.mask['CLM']
    """
    def __call__(self, eopatch):
        return eopatch.mask['IS_DATA'].astype(bool) & np.logical_not(eopatch.mask['CLM'].astype(bool))

    
class AddValidCountTask(EOTask):
    """
    The task counts number of valid observations in time-series and stores the results in the timeless mask.
    """
    def __init__(self, count_what, feature_name):
        self.what = count_what
        self.name = feature_name

    def execute(self, eopatch):
        eopatch[(FeatureType.MASK_TIMELESS, self.name)] = np.count_nonzero(eopatch.mask[self.what], axis=0)
        return eopatch
    
    
class ValidDataFractionPredicate:
    """ Predicate that defines if a frame from EOPatch's time-series is valid or not. Frame is valid if the
    valid data fraction is above the specified threshold.
    """
    def __init__(self, threshold):
        self.threshold = threshold

    def __call__(self, array):
        coverage = np.sum(array.astype(np.uint8)) / np.prod(array.shape)
        return coverage > self.threshold

class NanDataPredicate:
    """ Predicate that defines if a frame from EOPatch's time-series contains nans --> invalid """
    def __init__(self):
        pass

    def __call__(self, array):
        nancount = np.sum(np.isnan(array))
        return nancount==0

def weighting_function(pix_size: int, median_pix_size: int, highest_weight_pix_size: int = 35,
                       skewness: int = 15) -> float:
    """ Creates weight to be applied to a parcel depending on its number of pixels (after pixelation) """
    if pix_size >= median_pix_size:
        return 1
    
    xs = np.linspace(1, median_pix_size, median_pix_size)
    y1 = skewnorm.pdf(xs, skewness, loc=highest_weight_pix_size-100/3.14, scale=100)
    y1 = y1 / max(y1)
    y1 = y1 + 1

    return y1[int(pix_size)].astype(np.float)


class AddWeightMapTask(EOTask):
    """ Computes the weight map used to compute the validation metric """

    def __init__(self, 
                 cultivated_feature: Tuple[FeatureType, str], 
                 not_declared_feature: Tuple[FeatureType, str], 
                 weight_feature: Tuple[FeatureType, str], 
                 radius: int = 2, seed: int = 4321):
        self.cultivated_feature = cultivated_feature
        self.not_declared_feature = not_declared_feature
        self.weight_feature = weight_feature
        self.radius = radius
        self.seed = seed
        
    def execute(self, eopatch: EOPatch) -> EOPatch:
        cultivated = eopatch[self.cultivated_feature].astype(np.uint8).squeeze()
        not_declared = eopatch[self.not_declared_feature].squeeze()

        np.random.seed(self.seed)

        # compute connected components on binary mask
        conn_comp = measure.label(cultivated, background=0)
        # number of connected components
        n_comp = np.max(conn_comp) + 1

        # Placeholder for outputs
        height, width = cultivated.shape
        weights = np.zeros((height, width), dtype=np.float32)
        contours = np.zeros((height, width), dtype=np.float32)
        counts = np.zeros((height, width), dtype=np.uint8)

        # Loop over connected components, ignoring background
        for ncc in tqdm(np.arange(1, n_comp)):
            parcel_mask = conn_comp == ncc
            # number of pixels of each component, i.e. parcel
            n_pixels = np.sum(parcel_mask)

            # compute external boundary of parcel 
            dilated_mask = binary_dilation(parcel_mask, selem=disk(radius=self.radius))
            contour = np.logical_and(~parcel_mask, dilated_mask)

            weight = weighting_function(n_pixels, median_pix_size=400)

            weights[parcel_mask] = weight
            contours += 2 * weight * contour
            # In case countours overlap, the average weight is taken
            counts += contour

        # combine weights from all parcels into a single map. First add (averaged) contours,
        # then weighted parcels, then background 
        weight_map = np.zeros((height, width), dtype=np.float32)
        weight_map[contours > 0] = contours[contours > 0] / counts[contours > 0]
        weight_map[weights > 0] = weights[weights > 0]
        weight_map[weight_map == 0] = 1

        # add zero weights at border and undeclared parcels
        weight_map[not_declared == True] = 0
        weight_map[:1, :] = 0
        weight_map[:, :1] = 0
        weight_map[-2:, :] = 0
        weight_map[:, -2:] = 0

        eopatch[self.weight_feature] = weight_map[..., np.newaxis]
        
        return eopatch

class PredictPatchTask(EOTask):
    """
    https://eo-learn.readthedocs.io/en/latest/examples/land-cover-map/SI_LULC_pipeline.html#6.-Model-construction-and-training
    Task to make model predictions on a patch. Provide the model 
    """
    def __init__(self, model, features_feature, args):
        self.model = model
        self.features_feature = features_feature
        self.args = args
    
    def execute(self, eopatch):
        pred_eopatch = EOPatch(bbox=eopatch.bbox)
        band_names = ['B01','B02','B03','B04','B05','B06','B07','B08','B8A','B09','B11','B12']
        tidx = list(range((self.args.n_time_frames+1)//2)) + list(range(-1*(self.args.n_time_frames//2), 0))
        print(f'selecting the first N//2 and the last N//2 time stamps: {tidx}')

        print(self.args.indices)

        x = []
        for ix in tidx: # outer most group: time index
            print(f'Time index {ix}')
            for band in self.args.bands:
                band_ix = band_names.index(band)
                if len(eopatch.data['BANDS']) > ix:
                    xx = eopatch.data['BANDS'][ix][:, :, band_ix]
                else:
                    xx = eopatch.data['BANDS'][0][:, :, band_ix]
                x.append(xx.astype(np.float32))
            for index in self.args.indices:
                if len(eopatch.data['BANDS']) > ix:
                    xx = eopatch.data[index][ix]
                else:
                    xx = eopatch.data[index][0]
                x.append(xx.astype(np.float32).squeeze())
                print(f'Normalized index {index} attached')
        x = np.expand_dims(np.stack(x), axis=0)
        x = torch.tensor(x.astype(np.float32))
        print('Input shape: ', x.shape)

        with torch.no_grad():
            prediction = self.model(x)
        print('Output shape: ', prediction.shape)
        # reshape to expected output shape
        prediction = prediction.numpy().squeeze()
        prediction = prediction[:, :, np.newaxis]
        prediction = np.round(prediction).astype(np.uint8)
        pred_eopatch[(FeatureType.MASK_TIMELESS, 'PREDICTION')] = prediction
        return pred_eopatch

# Preprocessing

We document our preprocessing pipeline, that was executed for the train and validation dataset prior to training the network. We use 90 EOPatches for training and 10 for validation, randomly selected. The following steps are performed:
* Add features NDVI,  NDWI and NDBI
* Remove images with cloud cover > 1%
* Remove missing data (NaNs)
* Add weight maps for MCC calculation

The code is found in `./ai4eo/preprocessing.py`. To reproduce our preprocessing, execute

`python preprocessing.py --raw-data-dir <yourpath>/eopathes/ --target-dir <yourpath>/dev_data/no_nans_no_clouds/ --cloud-threshold 0.99`

# Model Definition

The next cell contains our version of the SRResNet. It is adapted for our application from https://github.com/sgrvinod/a-PyTorch-Tutorial-to-Super-Resolution and implements the image superresolution network of https://arxiv.org/abs/1609.04802. The hyperparameter optimization was included by us using NNI (https://nni.readthedocs.io/en/stable/). The network hyperparameters that were used for training our final submission model were:
- learning rate: 1e-4
- batch size: 12
- n-channels: 125 
- n-blocks: 49 
- bands: B02 B03 B04 B05 B06 B07 B08 B8A B11 B12 (all 10m and 20m spatial resolution bands)
- n-time-frames: 8 (first 4 frames and last 4 frames)
- indices: NDVI NDWI NDBI

In [ ]:
class ConvolutionalBlock(nn.Module):
    '''
    Convolutional block: Convolution, BatchNorm, Activation
    credits: https://github.com/sgrvinod/a-PyTorch-Tutorial-to-Super-Resolution/blob/master/models.py
    '''
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, batch_norm=False, activation=None):
        '''
        :param in_channels: number of input channels
        :param out_channels: number of output channels
        :param kernel_size: kernel size
        :param stride: stride
        :param batch_norm: include a BN layer?
        :param activation: Type of activation; None if none
        '''
        super(ConvolutionalBlock, self).__init__()

        if activation is not None:
            activation = activation.lower()
            assert activation in {'prelu', 'leakyrelu', 'tanh'}

        # container, that will hold the layers in this convolutional block
        layers = list()
        # convolutional layer
        layers.append(
                nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, 
                    stride=stride, padding=kernel_size // 2)
                )
        # batch normalization, if wanted
        if batch_norm is True:
            layers.append(nn.BatchNorm2d(num_features=out_channels))

        # activation layer, if wanted
        if activation == 'prelu':
            layers.append(nn.PReLU())
        elif activation == 'leakyrelu':
            layers.append(nn.LeakyReLU(0.2))
        elif activation == 'tanh':
            layers.append(nn.Tanh())

        # put together the convolutional block as a sequence of the layers
        self.conv_block = nn.Sequential(*layers)

    def forward(self, input_):
        '''
        Forward propagation

        :param input: input images, a tensor of size (N, in_channels, w, h)
        :return: output images, a tensor of size (N, out_channels, w, h)
        '''
        output = self.conv_block(input_) #(N, out_channels, w, h)
        return output

class SubPixelConvolutionalBlock(nn.Module):
    """
    A subpixel convolutional block, comprising convolutional, pixel-shuffle, and PReLU activation layers.
    """
    def __init__(self, args, scaling_factor=2):
        """
        :param kernel_size: kernel size of the convolution
        :param n_channels: number of input and output channels
        :param scaling_factor: factor to scale input images by (along both dimensions)
        """
        super(SubPixelConvolutionalBlock, self).__init__()
        
        # convolutional layer that increases the number of channels by scaling factor^2
        # followed by pixel shuffle and PReLU
        self.conv = nn.Conv2d(in_channels=args.n_channels, out_channels=args.n_channels * (scaling_factor ** 2),
                            kernel_size=args.small_kernel_size, padding=args.small_kernel_size // 2)
        
        # These additional channels are shuffled to form additional pixels
        # upscaling each dimension by the scaling factor
        self.pixel_shuffle = nn.PixelShuffle(upscale_factor=scaling_factor)
        self.prelu = nn.PReLU()

    def forward(self, input_):
        """
        Forward propagation.
        :param input: input images, a tensor of size (N, n_channels, w, h)
        :return: scaled output images, a tensor of size (N, n_channels, w * scaling factor, h * scaling factor)
        """
        output = self.conv(input_)  # (N, n_channels * scaling factor^2, w, h)
        output = self.pixel_shuffle(output)  # (N, n_channels, w * scaling factor, h * scaling factor)
        output = self.prelu(output)  # (N, n_channels, w * scaling factor, h * scaling factor)

        return output


class ResidualBlock(nn.Module):
    """
    A residual block, comprising two convolutional blocks with a residual connection across them.
    """
    def __init__(self, args):
        """
        :param kernel_size: kernel size
        :param n_channels: number of input and output channels (same because the input must be added to the output)
        """
        super(ResidualBlock, self).__init__()

        # first convolutional block
        self.conv_block1 = ConvolutionalBlock(in_channels=args.n_channels, out_channels=args.n_channels, 
                kernel_size=args.small_kernel_size, batch_norm=True, activation='PReLu')

        # second convolutional block
        self.conv_block2 = ConvolutionalBlock(in_channels=args.n_channels, out_channels=args.n_channels,
                kernel_size=args.small_kernel_size, batch_norm=True, activation=None)

    def forward(self, input_):
        """
        Forward propagation
        :param input: input images, a tensor of size (N, n_channels, w, h)
        :return: output images, a tensor of size (N, n_channels, w, h)
        """
        residual = input_ # (N, n_channels, w, h)
        output = self.conv_block1(input_) # (N, n_channels, w, h)
        output = self.conv_block2(output) # (N, n_channels, w, h)
        output = output + residual # (N, n_channels, w, h)

        return output
    

class SRResNet(nn.Module):
    '''
    The SRResNet, as defined in the paper.
    credits: https://github.com/sgrvinod/a-PyTorch-Tutorial-to-Super-Resolution/blob/master/models.py
    '''
    def __init__(self, args):
        """
        :param large_kernel_size: kernel size of the first and last convolutions which transform the inputs and outputs
        :param small_kernel_size: kernel size of all convolutions in-between, i.e. those in the residual and subpixel convolutional blocks
        :param n_channels: number of channels in-between, i.e. the input and output channels for the residual and subpixel convolutional blocks
        :param n_blocks: number of residual blocks
        :param scaling_factor: factor to scale input images by (along both dimensions) in the subpixel convolutional block
        """
        self.args = args
        super(SRResNet, self).__init__()

        # Scaling factor must be 2, 4 or 8
        scaling_factor = int(args.scaling_factor)
        assert scaling_factor in {2, 4, 8}, "The scaling factor must be 2, 4, or 8!"

        # First convolutional block
        self.conv_block1 = ConvolutionalBlock(in_channels=args.input_channels, out_channels=args.n_channels, 
                kernel_size=args.large_kernel_size,
                batch_norm=False, activation='PReLu')

        # Sequence of residual blocks
        self.residual_blocks = nn.Sequential(
                *[ResidualBlock(args) for i in range(args.n_blocks)]
                )

        # Another convolutional block
        self.conv_block2 = ConvolutionalBlock(in_channels=args.n_channels, out_channels=args.n_channels,
                kernel_size=args.small_kernel_size, batch_norm=True, activation=None)

        # Upscaling: by sub-pixel convolution, each such block upscaling by a factor of 2
        n_subpixel_convolutional_blocks = int(math.log2(args.scaling_factor))
        print(f'times subpixel: {n_subpixel_convolutional_blocks}')
        self.subpixel_convolutional_blocks = nn.Sequential(
                *[SubPixelConvolutionalBlock(args, scaling_factor=2) for i in range(n_subpixel_convolutional_blocks)]
                )

        # Last convolutional block
        self.conv_block3 = ConvolutionalBlock(in_channels=args.n_channels, out_channels=1,
                kernel_size=args.large_kernel_size, batch_norm=False, activation='Tanh')

        # Final sigmoid layer
        self.sigmoid = nn.Sigmoid()

    def forward(self, lr_imgs):
        """
        Forward propagation

        :param lr_imgs: low-resolution input images, a tensor of size (N, 3, w, h)
        :return: super-resolution output images, a tensor of size (N, 3, w * scaling factor, h * scaling factor)
        """
        output = self.conv_block1(lr_imgs) # (N, input_channels, w, h)
        residual = output # (N, n_channels, w, h)
        output = self.residual_blocks(output) # (N, n_channels, w, h)
        output =self.conv_block2(output) # (N, n_channels, w, h)
        output = output + residual
        output = self.subpixel_convolutional_blocks(output) # (N, n_channels, w * scaling factor, h * scaling factor)
        sr_imgs = self.conv_block3(output) # (N, 1, w * scaling factor, h * scaling factor)
        sr_imgs = self.sigmoid(sr_imgs)
        return sr_imgs

    def get_device(self):
        """Return gpu if available, else cpu"""
        if torch.cuda.is_available():
            return 'cuda:0'
        else:
            return 'cpu'


**Training Details:**

We trained the model on a single NVIDIA A100 GPU, using 1000 random cropped images of 32 x 32 pixel dimension. Image augmentation is applied, each image is rotated by 90, 180, and 270 degrees in addition to the standard orientation. One epoch of training takes about 50 minutes. We trained for 100 epochs. This resulted in a weighted **MCC score of 0.831**.

The code is found in `./ai4eo/model.py`. To reproduce our model training, execute

`python model.py --processed-data-dir <yourpath>/dev_data/no_nans_no_clouds/ --s2-random --n-s2 1000 --max-epochs 101 --learning-rate 1e-4 --batch-size 12 --n-channels 125 --n-blocks 49 --bands B02 B03 B04 B05 B06 B07 B08 B8A B11 B12  --n-time-frames 8 --checkpoint-epoch 20 --indices NDVI NDWI NDBI --patience 20 --target-dir <yourpath>`

# Make Predictions and create Submission

In the next cells the trained model is loaded and predictions are made. A submission is created by performing the necessary preprocessing steps and applying the model. The created images are exported as tiff files.

Create the submission, given the saved model and the following paramters. The only argument that should be adapted is

`args.raw_data_dir`

In [ ]:
args = argparse.Namespace()
args.fixed_random_seed = True
args.trained_model = "best_model.pt"
args.target_dir = 'submission'
args.n_processes = 1 # Parallel processes for EOExecutor, on our GPU only 1 was possible
args.overwrite = True # overwrite the output files
args.flag = 'test' # Make predictions on the test set
args.cloud_threshold = 0.999 # Filter all cloudy scenes exceeding 0.1% of cloud coverage

args.n_time_frames = 8 # number of time frames
args.s2_length = 500 # low res image side length
args.batch_size = 1 # make predictions one at a time
args.scaling_factor = 4 # low res to high res scaling factor
args.n_channels = 125 # SRResNet channels
args.bands = ['B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B8A', 'B11', 'B12'] # Sentinel bands
args.indices = ["NDVI", "NDWI", "NDBI"]
args.large_kernel_size = 9
args.small_kernel_size = 3
args.n_blocks = 49
args.learning_rate = 1e-3 # parameter not used in eval mode, but it needs to be present

In [ ]:
# Our submission function expects a directory structure
# <args.raw_datadir>/test/eopatch-00
# etc for the test set eopatches
args.raw_data_dir = '/test_data/eopatches/' # change this to your local test patch path. 

Load the provided model that is already trained. On CPU, making one prediction took about 2:30 minutes.

In [ ]:
if torch.cuda.is_available():
    model_state = torch.load(args.trained_model)
else:
    model_state = torch.load(args.trained_model, map_location=torch.device('cpu'))
        
args.input_channels = args.n_time_frames*(len(args.bands)+len(args.indices))
eomodel = SRResNet(args)
eomodel.load_state_dict(model_state)

In [ ]:
def create_submission(args, eomodel):

    # --------------------
    # INPUT CHECK
    # --------------------

    # for compatibility with the args namespace of the SRResNet, we provide
    # the default arguments here
    assert args.s2_length==500
    assert args.batch_size==1

    # --------------------
    # USEFUL DEFINITIONS
    # --------------------

    # band specification
    # https://sentinels.copernicus.eu/web/sentinel/user-guides/sentinel-2-msi/resolutions/spatial
    band_names = ['B01','B02','B03','B04','B05','B06','B07','B08','B8A','B09','B11','B12'] # from starter notebook
    band_wavelength = [443, 490, 560, 665, 705, 740, 783, 842, 865, 940, 1610, 2190] # nm
    band_spatial_resolution = [60, 10, 10, 10, 20, 20, 20, 10, 20, 60, 20, 20] # m

    # EOPatches for input
    eops_train = os.listdir(args.raw_data_dir)

    # --------------------
    # DEFINE THE WORKFLOW
    # --------------------

    # load eopatches from raw data
    load_task = LoadTask(path=args.raw_data_dir)

    # compute reflectances
    compute_reflectances = ComputeReflectances((FeatureType.DATA, 'BANDS'))

    # add the features: NDVI, NDWI, NDBI 
    ndvi = NormalizedDifferenceIndexTask((FeatureType.DATA, 'BANDS'), (FeatureType.DATA, 'NDVI'),
                                     [band_names.index('B08'), band_names.index('B04')])
    ndwi = NormalizedDifferenceIndexTask((FeatureType.DATA, 'BANDS'), (FeatureType.DATA, 'NDWI'),
                                     [band_names.index('B03'), band_names.index('B08')])
    ndbi = NormalizedDifferenceIndexTask((FeatureType.DATA, 'BANDS'), (FeatureType.DATA, 'NDBI'),
                                     [band_names.index('B11'), band_names.index('B08')])

    # validity masks
    # Validate pixels using SentinelHub's cloud detection mask and region of acquisition
    add_sh_validmask = AddValidDataMaskTask(SentinelHubValidData(), 'IS_VALID')

    # Count the number of valid observations per pixel using valid data mask
    add_valid_count = AddValidCountTask('IS_VALID', 'VALID_COUNT')

    # Filter out cloudy scenes
    valid_data_predicate = ValidDataFractionPredicate(args.cloud_threshold)
    filter_task = SimpleFilterTask((FeatureType.MASK, 'IS_VALID'), valid_data_predicate)

    # Filter out nans
    nan_data_predicate = NanDataPredicate()
    nan_filter_task = SimpleFilterTask((FeatureType.DATA, 'NDVI'), nan_data_predicate)

    # Predict
    predict_task = PredictPatchTask(eomodel, (FeatureType.DATA, 'BANDS'), args)
        
    # EXPORT PREDICTION AS TIFF - copied from starter notebook
    # Export the predicted binary mask as tiff for submission
    # NOTE: make sure both 0s and 1s are correctly exported
    export_task = ExportToTiff(feature=(FeatureType.MASK_TIMELESS, 'PREDICTION'),
                          folder=args.target_dir, crs='epsg:32633', image_dtype=np.uint8, no_data_value=255)

    save_task = SaveTask(path=args.target_dir, overwrite_permission=True)

    # construct the graph
    workflow = LinearWorkflow(load_task,
                              compute_reflectances,
                              ndvi,
                              ndwi,
                              ndbi,
                              add_sh_validmask,
                              add_valid_count,
                              filter_task,
                              nan_filter_task,
                              predict_task,
                              #save_task,
                              export_task
                              )

    # --------------------
    # EXECUTE THE WORKFLOW
    # --------------------

    # Define additional parameters of the workflow
    execution_args = []

    # need to specify the files that should be loaded 
    # loop all available eopatches and add each to the argument list
    execution_args = []

    eops_test = sorted(os.listdir(f'{args.raw_data_dir}/{args.flag}/'))

    for eop_test in eops_test:
        eop_exec_args = {
                load_task:   {'eopatch_folder': f'{args.flag}/{eop_test}'},
                save_task:   {'eopatch_folder': f'highres_{eop_test}'},
                export_task: {'filename': f'{eop_test}.tif'}
                        }
                                
        execution_args.append(eop_exec_args)

    # Execute the workflow
    executor = EOExecutor(workflow, execution_args, save_logs=True)
    executor.run(workers=args.n_processes, multiprocess=True)

    executor.make_report()
    print('Report was saved to location: {}'.format(executor.get_report_filename()))

    failed_ids = executor.get_failed_executions()
    if failed_ids:
        raise RuntimeError(f'Execution failed EOPatches with IDs:\n{failed_ids}\n'
                           f'For more info check report at {executor.get_report_filename()}')


Execute this cell to make the predictions on the test set (This can take some time).

In [ ]:
create_submission(args, eomodel)

In [ ]:
os.listdir('./submission') # list the TIFFs that were created

Visualize the predictions by comparing RGB images and predicted high resolution maps of cultivated land. *Left column:* earliest cloud-free scene in the growing season. *Center column:* latest cloud-free scene in the growing season. *Right column:* Predicted high resolution map of cultivated land

In [ ]:
r_ = len(os.listdir(args.target_dir))
c_ = 3

eops_test = sorted(os.listdir(f'{args.raw_data_dir}/{args.flag}/'))
tiff_test = sorted(os.listdir(args.target_dir))

fig, axs = plt.subplots(figsize=(4*c_, 4*r_), nrows=r_, ncols=c_)

for i, tp, tt in zip(range(r_), eops_test, tiff_test):
    eopatch = EOPatch.load(os.path.join(f'{args.raw_data_dir}/{args.flag}/', tp))
    
    vis_factor = 3.5

    norm_factor = eopatch.scalar['NORM_FACTORS'][0]

    
    valid_data = np.mean(eopatch.mask['IS_DATA'] & ~eopatch.mask['CLM'], axis=(1,2,3))
    
    tidx = np.where(valid_data>args.cloud_threshold)[0][0] # earliest cloud free scene in growing season
    
    axs[i,0].imshow(vis_factor * norm_factor * eopatch.data['BANDS'][tidx][..., [3, 2, 1]])
    axs[i,0].set_title(f'S2 L2A - {eopatch.timestamp[tidx]}')
    
    tidx = np.where(valid_data>args.cloud_threshold)[0][-1] # latest cloud free scene in growing season
    
    axs[i,1].imshow(vis_factor * norm_factor * eopatch.data['BANDS'][tidx][..., [3, 2, 1]])
    axs[i,1].set_title(f'S2 L2A - {eopatch.timestamp[tidx]}')
    
    try:
        img = Image.open(os.path.join(args.target_dir, tt))
        x_img = np.array(img)
    except:
        continue

    axs[i,2].imshow(x_img)
    axs[i,2].set_title('Predicted map')

fig.tight_layout()